In [1]:


!pip install -q gradio
!pip install -q requests
!pip install -q beautifulsoup4
!pip install -q google-generativeai
!pip install -q langchain
!pip install -q chromadb
!pip install -q selenium
!pip install -q webdriver-manager
!pip install -q pillow
!pip install -q python-dotenv
!pip install -q validators
!pip install -q contrast-ratio

# Import all necessary libraries
import os
import re
import json
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import gradio as gr
import google.generativeai as genai
from PIL import Image, ImageColor
import validators
import warnings
warnings.filterwarnings('ignore')

GEMINI_API_KEY = ""
genai.configure(api_key=GEMINI_API_KEY)

print("✅ All dependencies installed successfully!")
print("🔑 Remember to add your Gemini API key in the GEMINI_API_KEY variable above")

ERROR: Could not find a version that satisfies the requirement contrast-ratio (from versions: none)
ERROR: No matching distribution found for contrast-ratio
✅ All dependencies installed successfully!
🔑 Remember to add your Gemini API key in the GEMINI_API_KEY variable above


In [2]:
# AI-Powered Site Auditor - Section 2: Website Scraper and Content Analyzer

class WebsiteScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def fetch_website_content(self, url):
        """Fetch and parse website content"""
        try:
            if not validators.url(url):
                return None, "Invalid URL format"

            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract key elements
            content_data = {
                'url': url,
                'title': soup.find('title').get_text() if soup.find('title') else 'No title',
                'html_content': str(soup),
                'meta_tags': self.extract_meta_tags(soup),
                'images': self.extract_images(soup, url),
                'forms': self.extract_forms(soup),
                'links': self.extract_links(soup, url),
                'scripts': self.extract_scripts(soup),
                'css_styles': self.extract_css_styles(soup),
                'headings': self.extract_headings(soup),
                'content_text': soup.get_text()[:5000]  # First 5000 chars
            }

            return content_data, None

        except requests.RequestException as e:
            return None, f"Error fetching website: {str(e)}"
        except Exception as e:
            return None, f"Error parsing website: {str(e)}"

    def extract_meta_tags(self, soup):
        """Extract meta tags information"""
        meta_tags = {}
        for meta in soup.find_all('meta'):
            name = meta.get('name') or meta.get('property') or meta.get('http-equiv')
            content = meta.get('content')
            if name and content:
                meta_tags[name] = content
        return meta_tags

    def extract_images(self, soup, base_url):
        """Extract image information"""
        images = []
        for img in soup.find_all('img'):
            img_data = {
                'src': urljoin(base_url, img.get('src', '')),
                'alt': img.get('alt', ''),
                'title': img.get('title', ''),
                'width': img.get('width', ''),
                'height': img.get('height', '')
            }
            images.append(img_data)
        return images

    def extract_forms(self, soup):
        """Extract form information"""
        forms = []
        for form in soup.find_all('form'):
            form_data = {
                'action': form.get('action', ''),
                'method': form.get('method', 'GET'),
                'inputs': []
            }
            for input_tag in form.find_all(['input', 'textarea', 'select']):
                input_data = {
                    'type': input_tag.get('type', 'text'),
                    'name': input_tag.get('name', ''),
                    'required': input_tag.has_attr('required'),
                    'label': self.find_label_for_input(soup, input_tag)
                }
                form_data['inputs'].append(input_data)
            forms.append(form_data)
        return forms

    def find_label_for_input(self, soup, input_tag):
        """Find associated label for input field"""
        input_id = input_tag.get('id')
        if input_id:
            label = soup.find('label', {'for': input_id})
            if label:
                return label.get_text().strip()
        return ''

    def extract_links(self, soup, base_url):
        """Extract link information"""
        links = []
        for link in soup.find_all('a', href=True):
            link_data = {
                'href': urljoin(base_url, link['href']),
                'text': link.get_text().strip(),
                'title': link.get('title', ''),
                'target': link.get('target', '')
            }
            links.append(link_data)
        return links

    def extract_scripts(self, soup):
        """Extract script information"""
        scripts = []
        for script in soup.find_all('script'):
            script_data = {
                'src': script.get('src', ''),
                'type': script.get('type', ''),
                'content': script.string[:200] if script.string else ''  # First 200 chars
            }
            scripts.append(script_data)
        return scripts

    def extract_css_styles(self, soup):
        """Extract CSS style information"""
        styles = []
        for style in soup.find_all('style'):
            if style.string:
                styles.append(style.string[:500])  # First 500 chars
        return styles

    def extract_headings(self, soup):
        """Extract heading structure"""
        headings = []
        for level in range(1, 7):
            for heading in soup.find_all(f'h{level}'):
                headings.append({
                    'level': level,
                    'text': heading.get_text().strip(),
                    'id': heading.get('id', '')
                })
        return headings

# Initialize the scraper
scraper = WebsiteScraper()

print("✅ Website Scraper initialized successfully!")

✅ Website Scraper initialized successfully!


In [3]:
# AI-Powered Site Auditor - Section 3: Compliance Checker and Rules Engine

class ComplianceChecker:
    def __init__(self):
        self.compliance_rules = {
            'gdpr': self.check_gdpr_compliance,
            'accessibility': self.check_accessibility_compliance,
            'wcag': self.check_wcag_compliance,
            'seo': self.check_seo_compliance,
            'security': self.check_security_compliance
        }

    def analyze_compliance(self, website_data):
        """Run all compliance checks"""
        results = {
            'url': website_data['url'],
            'title': website_data['title'],
            'issues': [],
            'recommendations': [],
            'score': 0,
            'categories': {}
        }

        for category, check_function in self.compliance_rules.items():
            category_results = check_function(website_data)
            results['categories'][category] = category_results
            results['issues'].extend(category_results['issues'])
            results['recommendations'].extend(category_results['recommendations'])

        # Calculate overall score
        total_checks = sum(len(cat['issues']) + len(cat['passed']) for cat in results['categories'].values())
        passed_checks = sum(len(cat['passed']) for cat in results['categories'].values())
        results['score'] = round((passed_checks / total_checks * 100) if total_checks > 0 else 0, 1)

        return results

    def check_gdpr_compliance(self, data):
        """Check GDPR compliance requirements"""
        issues = []
        recommendations = []
        passed = []

        html_content = data['html_content'].lower()

        # Check for cookie consent
        cookie_keywords = ['cookie', 'consent', 'gdpr', 'privacy policy', 'data protection']
        has_cookie_consent = any(keyword in html_content for keyword in cookie_keywords)

        if not has_cookie_consent:
            issues.append({
                'severity': 'High',
                'category': 'GDPR',
                'issue': 'Missing cookie consent mechanism',
                'description': 'No cookie consent banner or privacy controls detected'
            })
            recommendations.append({
                'category': 'GDPR',
                'recommendation': 'Add a cookie consent banner that allows users to accept/reject cookies',
                'priority': 'High'
            })
        else:
            passed.append('Cookie consent mechanism present')

        # Check for privacy policy
        privacy_links = [link for link in data['links'] if 'privacy' in link['text'].lower() or 'privacy' in link['href'].lower()]
        if not privacy_links:
            issues.append({
                'severity': 'High',
                'category': 'GDPR',
                'issue': 'Missing privacy policy link',
                'description': 'No privacy policy link found on the website'
            })
            recommendations.append({
                'category': 'GDPR',
                'recommendation': 'Add a clearly visible link to your privacy policy',
                'priority': 'High'
            })
        else:
            passed.append('Privacy policy link present')

        # Check for data collection forms without consent
        for form in data['forms']:
            has_email_input = any(inp['type'] == 'email' for inp in form['inputs'])
            has_consent_checkbox = any('consent' in inp['name'].lower() or 'privacy' in inp['name'].lower() for inp in form['inputs'])

            if has_email_input and not has_consent_checkbox:
                issues.append({
                    'severity': 'Medium',
                    'category': 'GDPR',
                    'issue': 'Form collecting email without explicit consent',
                    'description': 'Email collection form lacks consent checkbox'
                })
                recommendations.append({
                    'category': 'GDPR',
                    'recommendation': 'Add a consent checkbox to forms collecting personal data',
                    'priority': 'Medium'
                })

        return {'issues': issues, 'recommendations': recommendations, 'passed': passed}

    def check_accessibility_compliance(self, data):
        """Check accessibility compliance (ADA/WCAG)"""
        issues = []
        recommendations = []
        passed = []

        # Check for missing alt text
        images_without_alt = [img for img in data['images'] if not img['alt']]
        if images_without_alt:
            issues.append({
                'severity': 'High',
                'category': 'Accessibility',
                'issue': f'{len(images_without_alt)} images missing alt text',
                'description': 'Images without alt text are not accessible to screen readers'
            })
            recommendations.append({
                'category': 'Accessibility',
                'recommendation': 'Add descriptive alt text to all images',
                'priority': 'High'
            })
        else:
            passed.append('All images have alt text')

        # Check for proper heading structure
        headings = data['headings']
        if headings:
            # Check if starts with h1
            first_heading = min(headings, key=lambda x: x['level'])
            if first_heading['level'] != 1:
                issues.append({
                    'severity': 'Medium',
                    'category': 'Accessibility',
                    'issue': 'Page does not start with H1',
                    'description': 'Proper heading hierarchy should start with H1'
                })
                recommendations.append({
                    'category': 'Accessibility',
                    'recommendation': 'Ensure page has a proper H1 heading as the main title',
                    'priority': 'Medium'
                })
            else:
                passed.append('Proper heading structure with H1')

        # Check for form labels
        for form in data['forms']:
            inputs_without_labels = [inp for inp in form['inputs'] if not inp['label'] and inp['type'] not in ['submit', 'button', 'hidden']]
            if inputs_without_labels:
                issues.append({
                    'severity': 'High',
                    'category': 'Accessibility',
                    'issue': f'Form inputs without labels detected',
                    'description': 'Form inputs need associated labels for screen readers'
                })
                recommendations.append({
                    'category': 'Accessibility',
                    'recommendation': 'Add proper labels to all form inputs',
                    'priority': 'High'
                })

        # Check for skip navigation
        soup = BeautifulSoup(data['html_content'], 'html.parser')
        skip_nav = soup.find('a', string=re.compile(r'skip.*nav', re.I))
        if not skip_nav:
            issues.append({
                'severity': 'Medium',
                'category': 'Accessibility',
                'issue': 'Missing skip navigation link',
                'description': 'Skip navigation helps keyboard users bypass repetitive content'
            })
            recommendations.append({
                'category': 'Accessibility',
                'recommendation': 'Add a "Skip to main content" link at the beginning of the page',
                'priority': 'Medium'
            })
        else:
            passed.append('Skip navigation link present')

        return {'issues': issues, 'recommendations': recommendations, 'passed': passed}

    def check_wcag_compliance(self, data):
        """Check WCAG specific guidelines"""
        issues = []
        recommendations = []
        passed = []

        # Check for language attribute
        soup = BeautifulSoup(data['html_content'], 'html.parser')
        html_tag = soup.find('html')
        if not html_tag or not html_tag.get('lang'):
            issues.append({
                'severity': 'Medium',
                'category': 'WCAG',
                'issue': 'Missing language attribute',
                'description': 'HTML tag should have a lang attribute for screen readers'
            })
            recommendations.append({
                'category': 'WCAG',
                'recommendation': 'Add lang attribute to HTML tag (e.g., <html lang="en">)',
                'priority': 'Medium'
            })
        else:
            passed.append('Language attribute present')

        # Check for page title
        if not data['title'] or data['title'] == 'No title':
            issues.append({
                'severity': 'High',
                'category': 'WCAG',
                'issue': 'Missing page title',
                'description': 'Page title is essential for screen readers and SEO'
            })
            recommendations.append({
                'category': 'WCAG',
                'recommendation': 'Add a descriptive title tag to the page',
                'priority': 'High'
            })
        else:
            passed.append('Page title present')

        # Check for focus indicators (basic check)
        css_content = ' '.join(data['css_styles']).lower()
        has_focus_styles = ':focus' in css_content
        if not has_focus_styles:
            issues.append({
                'severity': 'Medium',
                'category': 'WCAG',
                'issue': 'No custom focus indicators detected',
                'description': 'Custom focus styles improve keyboard navigation visibility'
            })
            recommendations.append({
                'category': 'WCAG',
                'recommendation': 'Add visible focus indicators for interactive elements',
                'priority': 'Medium'
            })
        else:
            passed.append('Focus indicators present')

        return {'issues': issues, 'recommendations': recommendations, 'passed': passed}

    def check_seo_compliance(self, data):
        """Check basic SEO compliance"""
        issues = []
        recommendations = []
        passed = []

        # Check meta description
        meta_description = data['meta_tags'].get('description', '')
        if not meta_description:
            issues.append({
                'severity': 'Medium',
                'category': 'SEO',
                'issue': 'Missing meta description',
                'description': 'Meta description helps search engines understand page content'
            })
            recommendations.append({
                'category': 'SEO',
                'recommendation': 'Add a compelling meta description (150-160 characters)',
                'priority': 'Medium'
            })
        else:
            passed.append('Meta description present')

        # Check for multiple H1 tags
        h1_count = len([h for h in data['headings'] if h['level'] == 1])
        if h1_count > 1:
            issues.append({
                'severity': 'Low',
                'category': 'SEO',
                'issue': f'Multiple H1 tags found ({h1_count})',
                'description': 'Multiple H1 tags can confuse search engines'
            })
            recommendations.append({
                'category': 'SEO',
                'recommendation': 'Use only one H1 tag per page',
                'priority': 'Low'
            })
        elif h1_count == 1:
            passed.append('Single H1 tag present')

        return {'issues': issues, 'recommendations': recommendations, 'passed': passed}

    def check_security_compliance(self, data):
        """Check basic security compliance"""
        issues = []
        recommendations = []
        passed = []

        # Check if site is HTTPS
        if not data['url'].startswith('https://'):
            issues.append({
                'severity': 'High',
                'category': 'Security',
                'issue': 'Site not using HTTPS',
                'description': 'HTTPS is essential for secure data transmission'
            })
            recommendations.append({
                'category': 'Security',
                'recommendation': 'Implement SSL certificate and redirect HTTP to HTTPS',
                'priority': 'High'
            })
        else:
            passed.append('HTTPS encryption enabled')

        # Check for mixed content
        soup = BeautifulSoup(data['html_content'], 'html.parser')
        http_resources = []
        for tag in soup.find_all(['img', 'script', 'link']):
            src = tag.get('src') or tag.get('href', '')
            if src.startswith('http://'):
                http_resources.append(src)

        if http_resources and data['url'].startswith('https://'):
            issues.append({
                'severity': 'Medium',
                'category': 'Security',
                'issue': f'Mixed content detected ({len(http_resources)} resources)',
                'description': 'HTTP resources on HTTPS pages can cause security warnings'
            })
            recommendations.append({
                'category': 'Security',
                'recommendation': 'Update all resource URLs to use HTTPS',
                'priority': 'Medium'
            })

        return {'issues': issues, 'recommendations': recommendations, 'passed': passed}

# Initialize the compliance checker
compliance_checker = ComplianceChecker()

print("✅ Compliance Checker initialized successfully!")

✅ Compliance Checker initialized successfully!


In [4]:
# AI-Powered Site Auditor - Section 4: AI-Powered Analysis with Gemini

class AIAnalyzer:
    def __init__(self, api_key):
        if not api_key:
            raise ValueError("Gemini API key is required")

        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def analyze_website_with_ai(self, website_data, compliance_results):
        """Use Gemini to provide intelligent analysis and recommendations"""
        try:
            # Prepare context for AI analysis
            context = self.prepare_analysis_context(website_data, compliance_results)

            # Generate AI analysis
            ai_insights = self.generate_ai_insights(context)

            # Generate priority recommendations
            priority_recommendations = self.generate_priority_recommendations(context)

            # Generate improvement suggestions
            improvement_suggestions = self.generate_improvement_suggestions(context)

            # Generate analysis summary
            analysis_summary = self.generate_analysis_summary(compliance_results)


            return {
                'ai_insights': ai_insights,
                'priority_recommendations': priority_recommendations,
                'improvement_suggestions': improvement_suggestions,
                'analysis_summary': analysis_summary
            }

        except Exception as e:
            return {
                'ai_insights': f"AI analysis failed: {str(e)}",
                'priority_recommendations': [],
                'improvement_suggestions': [],
                'analysis_summary': "AI analysis unavailable"
            }

    def prepare_analysis_context(self, website_data, compliance_results):
        """Prepare context for AI analysis"""
        context = {
            'url': website_data['url'],
            'title': website_data['title'],
            'total_issues': len(compliance_results['issues']),
            'compliance_score': compliance_results['score'],
            'issues_by_category': {},
            'website_structure': {
                'images_count': len(website_data['images']),
                'forms_count': len(website_data['forms']),
                'links_count': len(website_data['links']),
                'headings_count': len(website_data['headings'])
            }
        }

        # Group issues by category and severity
        for issue in compliance_results['issues']:
            category = issue['category']
            if category not in context['issues_by_category']:
                context['issues_by_category'][category] = {'High': 0, 'Medium': 0, 'Low': 0}
            context['issues_by_category'][category][issue['severity']] += 1

        return context

    def generate_ai_insights(self, context):
        """Generate AI-powered insights about the website"""
        prompt = f"""
        Analyze this website compliance audit data and provide intelligent insights:

        Website: {context['url']}
        Title: {context['title']}
        Compliance Score: {context['compliance_score']}%
        Total Issues: {context['total_issues']}

        Issues by Category:
        {json.dumps(context['issues_by_category'], indent=2)}

        Website Structure:
        - Images: {context['website_structure']['images_count']}
        - Forms: {context['website_structure']['forms_count']}
        - Links: {context['website_structure']['links_count']}
        - Headings: {context['website_structure']['headings_count']}

        Provide 3-4 key insights about the website's compliance status, focusing on:
        1. Overall compliance health
        2. Most critical areas needing attention
        3. Positive aspects that are working well
        4. Business impact of current issues

        Keep insights concise and actionable.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Failed to generate AI insights: {str(e)}"

    def generate_priority_recommendations(self, context):
        """Generate prioritized recommendations"""
        prompt = f"""
        Based on this website compliance audit, provide the top 5 priority recommendations:

        Website: {context['url']}
        Compliance Score: {context['compliance_score']}%
        Total Issues: {context['total_issues']}

        Issues by Category:
        {json.dumps(context['issues_by_category'], indent=2)}

        Provide exactly 5 recommendations in order of priority, considering:
        1. Legal compliance risk
        2. User experience impact
        3. Implementation difficulty
        4. Business impact

        Format each recommendation as:
        - Priority X: [Brief title] - [Concise description and why it's important]

        Focus on actionable items that can be implemented quickly.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Failed to generate priority recommendations: {str(e)}"

    def generate_improvement_suggestions(self, context):
        """Generate specific improvement suggestions"""
        prompt = f"""
        Provide specific, technical improvement suggestions for this website:

        Website: {context['url']}
        Compliance Score: {context['compliance_score']}%

        Issues by Category:
        {json.dumps(context['issues_by_category'], indent=2)}

        Provide concrete, implementable suggestions with code examples where appropriate.
        Focus on:
        1. GDPR compliance improvements
        2. Accessibility enhancements
        3. Technical implementation tips
        4. Tools and resources that can help

        Keep suggestions practical and include specific HTML/CSS examples when relevant.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Failed to generate improvement suggestions: {str(e)}"

    def generate_analysis_summary(self, compliance_results):
        """Generate a summary of the analysis"""
        total_issues = len(compliance_results['issues'])
        high_severity = len([i for i in compliance_results['issues'] if i['severity'] == 'High'])
        medium_severity = len([i for i in compliance_results['issues'] if i['severity'] == 'Medium'])
        low_severity = len([i for i in compliance_results['issues'] if i['severity'] == 'Low'])

        summary = f"""
        📊 **Analysis Summary**

        **Overall Score:** {compliance_results['score']}%
        **Total Issues Found:** {total_issues}

        **Issue Breakdown:**
        - 🔴 High Priority: {high_severity} issues
        - 🟡 Medium Priority: {medium_severity} issues
        - 🟢 Low Priority: {low_severity} issues

        **Categories Analyzed:**
        - GDPR Compliance
        - Accessibility (ADA/WCAG)
        - SEO Optimization
        - Security Best Practices
        """

        return summary

# Function to initialize AI analyzer (will be called when API key is provided)
def initialize_ai_analyzer(api_key):
    """Initialize the AI analyzer with API key"""
    if not api_key:
        return None
    try:
        return AIAnalyzer(api_key)
    except Exception as e:
        print(f"Failed to initialize AI analyzer: {str(e)}")
        return None

print("✅ AI Analyzer class ready!")
print("💡 AI analyzer will be initialized when you provide your Gemini API key")

✅ AI Analyzer class ready!
💡 AI analyzer will be initialized when you provide your Gemini API key


In [6]:
# AI-Powered Site Auditor - Section 5: RAG Module for Live Regulatory Content

import chromadb
from chromadb.config import Settings

class RAGModule:
    def __init__(self):
        self.client = chromadb.Client(Settings(
            chroma_db_impl="duckdb+parquet",
            persist_directory="./chroma_db"
        ))
        self.collection_name = "compliance_regulations"
        self.regulatory_sources = {
            'gdpr': 'https://gdpr.eu/what-is-gdpr/',
            'wcag': 'https://www.w3.org/WAI/WCAG21/quickref/',
            'ada': 'https://www.ada.gov/resources/web-guidance/'
        }
        self.initialize_collection()

    def initialize_collection(self):
        """Initialize or get the regulatory content collection"""
        try:
            self.collection = self.client.get_collection(self.collection_name)
            print("📚 Existing RAG collection loaded")
        except:
            self.collection = self.client.create_collection(
                name=self.collection_name,
                metadata={"description": "Compliance regulations and guidelines"}
            )
            self.populate_initial_content()
            print("📚 New RAG collection created and populated")

    def populate_initial_content(self):
        """Populate collection with initial regulatory content"""
        initial_content = [
            {
                'id': 'gdpr_cookies',
                'content': 'GDPR Article 7 requires clear and specific consent for cookie usage. Websites must provide users with the ability to withdraw consent as easily as giving it. Cookie consent banners should clearly explain what cookies are used for and allow granular control over cookie categories.',
                'source': 'GDPR',
                'category': 'cookies',
                'metadata': {'regulation': 'GDPR', 'article': '7', 'topic': 'consent'}
            },
            {
                'id': 'gdpr_privacy_policy',
                'content': 'Under GDPR Article 13, organizations must provide clear information about data processing through a privacy policy. This must include the purpose of processing, legal basis, data retention periods, and contact details of the data protection officer.',
                'source': 'GDPR',
                'category': 'privacy',
                'metadata': {'regulation': 'GDPR', 'article': '13', 'topic': 'transparency'}
            },
            {
                'id': 'wcag_alt_text',
                'content': 'WCAG 2.1 Success Criterion 1.1.1 requires that all non-text content has text alternatives that serve the equivalent purpose. Images must have alt attributes that describe the content and function of the image for screen reader users.',
                'source': 'WCAG',
                'category': 'accessibility',
                'metadata': {'standard': 'WCAG 2.1', 'criterion': '1.1.1', 'level': 'A'}
            },
            {
                'id': 'wcag_keyboard_navigation',
                'content': 'WCAG 2.1 Success Criterion 2.1.1 states that all functionality must be available from a keyboard. Interactive elements should be reachable and operable using only keyboard navigation, with visible focus indicators.',
                'source': 'WCAG',
                'category': 'accessibility',
                'metadata': {'standard': 'WCAG 2.1', 'criterion': '2.1.1', 'level': 'A'}
            },
            {
                'id': 'wcag_color_contrast',
                'content': 'WCAG 2.1 Success Criterion 1.4.3 requires a contrast ratio of at least 4.5:1 for normal text and 3:1 for large text against their background. This ensures text is readable for users with visual impairments.',
                'source': 'WCAG',
                'category': 'accessibility',
                'metadata': {'standard': 'WCAG 2.1', 'criterion': '1.4.3', 'level': 'AA'}
            },
            {
                'id': 'wcag_page_titles',
                'content': 'WCAG 2.1 Success Criterion 2.4.2 requires that web pages have titles that describe topic or purpose. Page titles should be descriptive and help users understand what page they are on.',
                'source': 'WCAG',
                'category': 'accessibility',
                'metadata': {'standard': 'WCAG 2.1', 'criterion': '2.4.2', 'level': 'A'}
            },
            {
                'id': 'ada_web_accessibility',
                'content': 'The Americans with Disabilities Act (ADA) applies to websites as places of public accommodation. Courts have increasingly held that websites must be accessible to people with disabilities, often referencing WCAG 2.1 Level AA as the standard.',
                'source': 'ADA',
                'category': 'legal',
                'metadata': {'law': 'ADA', 'standard_reference': 'WCAG 2.1 AA'}
            },
            {
                'id': 'gdpr_data_processing',
                'content': 'GDPR Article 6 establishes lawful bases for processing personal data. Consent must be freely given, specific, informed and unambiguous. Pre-ticked boxes or silence do not constitute valid consent.',
                'source': 'GDPR',
                'category': 'data_processing',
                'metadata': {'regulation': 'GDPR', 'article': '6', 'topic': 'lawful_basis'}
            },
            {
                'id': 'wcag_form_labels',
                'content': 'WCAG 2.1 Success Criterion 3.3.2 requires that labels or instructions are provided when content requires user input. Form inputs must have associated labels that clearly describe the required information.',
                'source': 'WCAG',
                'category': 'accessibility',
                'metadata': {'standard': 'WCAG 2.1', 'criterion': '3.3.2', 'level': 'A'}
            },
            {
                'id': 'gdpr_data_portability',
                'content': 'GDPR Article 20 grants individuals the right to data portability. Organizations must provide personal data in a structured, commonly used, and machine-readable format when requested.',
                'source': 'GDPR',
                'category': 'rights',
                'metadata': {'regulation': 'GDPR', 'article': '20', 'topic': 'data_portability'}
            }
        ]

        # Add content to collection
        documents = [item['content'] for item in initial_content]
        metadatas = [item['metadata'] for item in initial_content]
        ids = [item['id'] for item in initial_content]

        self.collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )

    def fetch_live_regulatory_content(self, source_type='gdpr'):
        """Fetch live content from regulatory websites"""
        try:
            url = self.regulatory_sources.get(source_type)
            if not url:
                return None

            response = requests.get(url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract relevant content based on source type
            if source_type == 'gdpr':
                return self.extract_gdpr_content(soup)
            elif source_type == 'wcag':
                return self.extract_wcag_content(soup)
            elif source_type == 'ada':
                return self.extract_ada_content(soup)

        except Exception as e:
            print(f"Failed to fetch {source_type} content: {str(e)}")
            return None

    def extract_gdpr_content(self, soup):
        """Extract GDPR-specific content"""
        content_sections = []

        # Look for article sections or important paragraphs
        for section in soup.find_all(['section', 'article', 'div'], class_=re.compile(r'content|article|section')):
            text = section.get_text().strip()
            if len(text) > 100 and any(keyword in text.lower() for keyword in ['consent', 'cookie', 'privacy', 'data']):
                content_sections.append({
                    'content': text[:500],  # Limit to 500 chars
                    'source': 'GDPR_Live',
                    'category': 'gdpr_update'
                })

        return content_sections[:5]  # Limit to 5 sections

    def extract_wcag_content(self, soup):
        """Extract WCAG-specific content"""
        content_sections = []

        # Look for guideline sections
        for section in soup.find_all(['section', 'div'], class_=re.compile(r'guideline|criterion|success')):
            text = section.get_text().strip()
            if len(text) > 100 and any(keyword in text.lower() for keyword in ['accessibility', 'screen reader', 'keyboard', 'contrast']):
                content_sections.append({
                    'content': text[:500],
                    'source': 'WCAG_Live',
                    'category': 'wcag_update'
                })

        return content_sections[:5]

    def extract_ada_content(self, soup):
        """Extract ADA-specific content"""
        content_sections = []

        # Look for guidance sections
        for section in soup.find_all(['section', 'div', 'p'], class_=re.compile(r'guidance|content')):
            text = section.get_text().strip()
            if len(text) > 100 and any(keyword in text.lower() for keyword in ['website', 'accessibility', 'disability', 'accommodation']):
                content_sections.append({
                    'content': text[:500],
                    'source': 'ADA_Live',
                    'category': 'ada_update'
                })

        return content_sections[:5]

    def update_regulatory_content(self):
        """Update collection with latest regulatory content"""
        try:
            for source_type in self.regulatory_sources.keys():
                print(f"Fetching {source_type.upper()} content...")
                new_content = self.fetch_live_regulatory_content(source_type)

                if new_content:
                    for i, content_item in enumerate(new_content):
                        doc_id = f"{source_type}_live_{i}_{int(time.time())}"

                        self.collection.add(
                            documents=[content_item['content']],
                            metadatas=[{
                                'source': content_item['source'],
                                'category': content_item['category'],
                                'updated': time.strftime('%Y-%m-%d')
                            }],
                            ids=[doc_id]
                        )

                    print(f"✅ Added {len(new_content)} {source_type.upper()} items")
                else:
                    print(f"⚠️ No new {source_type.upper()} content found")

        except Exception as e:
            print(f"Error updating regulatory content: {str(e)}")

    def query_regulatory_guidance(self, query, n_results=3):
        """Query the RAG system for relevant regulatory guidance"""
        try:
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results
            )

            if results['documents'] and results['documents'][0]:
                guidance = []
                for i, doc in enumerate(results['documents'][0]):
                    metadata = results['metadatas'][0][i] if results['metadatas'] else {}
                    guidance.append({
                        'content': doc,
                        'source': metadata.get('source', 'Unknown'),
                        'category': metadata.get('category', 'General'),
                        'relevance_score': 1 - results['distances'][0][i] if results['distances'] else 0
                    })
                return guidance
            else:
                return []

        except Exception as e:
            print(f"Error querying regulatory guidance: {str(e)}")
            return []

    def get_contextual_recommendations(self, compliance_issues):
        """Get contextual recommendations based on specific compliance issues"""
        recommendations = []

        for issue in compliance_issues:
            # Create query based on issue category and description
            query = f"{issue['category']} {issue['issue']} compliance requirements"

            guidance = self.query_regulatory_guidance(query, n_results=2)

            if guidance:
                recommendations.append({
                    'issue': issue['issue'],
                    'category': issue['category'],
                    'regulatory_guidance': guidance,
                    'implementation_priority': issue['severity']
                })

        return recommendations

# Initialize RAG module
try:
    rag_module = RAGModule()
    print("✅ RAG Module initialized successfully!")
    print("💡 Use rag_module.update_regulatory_content() to fetch latest regulations")
except Exception as e:
    print(f"⚠️ RAG Module initialization failed: {str(e)}")
    print("💡 RAG features will be limited without proper initialization")
    rag_module = None

⚠️ RAG Module initialization failed: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help!
💡 RAG features will be limited without proper initialization


In [7]:
# AI-Powered Site Auditor - Section 6: Report Generator and Formatting

class ReportGenerator:
    def __init__(self):
        self.severity_colors = {
            'High': '🔴',
            'Medium': '🟡',
            'Low': '🟢'
        }
        self.category_icons = {
            'GDPR': '🛡️',
            'Accessibility': '♿',
            'WCAG': '📋',
            'SEO': '🔍',
            'Security': '🔒'
        }

    def generate_comprehensive_report(self, website_data, compliance_results, ai_analysis=None, rag_recommendations=None):
        """Generate a comprehensive audit report"""
        report = {
            'executive_summary': self.generate_executive_summary(compliance_results),
            'detailed_findings': self.generate_detailed_findings(compliance_results),
            'priority_matrix': self.generate_priority_matrix(compliance_results),
            'implementation_roadmap': self.generate_implementation_roadmap(compliance_results),
            'technical_details': self.generate_technical_details(website_data, compliance_results)
        }

        if ai_analysis:
            report['ai_insights'] = ai_analysis

        if rag_recommendations:
            report['regulatory_guidance'] = rag_recommendations

        return report

    def generate_executive_summary(self, compliance_results):
        """Generate executive summary"""
        total_issues = len(compliance_results['issues'])
        high_issues = len([i for i in compliance_results['issues'] if i['severity'] == 'High'])
        score = compliance_results['score']

        # Determine risk level
        if score >= 80:
            risk_level = "✅ Low Risk"
            risk_description = "Website shows good compliance practices with minor issues to address."
        elif score >= 60:
            risk_level = "⚠️ Medium Risk"
            risk_description = "Website has several compliance gaps that should be addressed promptly."
        else:
            risk_level = "🚨 High Risk"
            risk_description = "Website has significant compliance issues requiring immediate attention."

        summary = f"""
## 📊 Executive Summary

**Website:** {compliance_results['url']}
**Audit Date:** {time.strftime('%Y-%m-%d %H:%M:%S')}
**Overall Compliance Score:** {score}%

### Risk Assessment
**Risk Level:** {risk_level}
{risk_description}

### Quick Stats
- **Total Issues Found:** {total_issues}
- **High Priority Issues:** {high_issues}
- **Categories Analyzed:** {len(compliance_results['categories'])}

### Key Findings
"""

        # Add category breakdown
        for category, results in compliance_results['categories'].items():
            issues_count = len(results['issues'])
            if issues_count > 0:
                icon = self.category_icons.get(category.upper(), '📌')
                summary += f"- {icon} **{category.upper()}:** {issues_count} issues found\n"

        return summary

    def generate_detailed_findings(self, compliance_results):
        """Generate detailed findings section"""
        findings = "## 🔍 Detailed Findings\n\n"

        for category, results in compliance_results['categories'].items():
            if results['issues']:
                icon = self.category_icons.get(category.upper(), '📌')
                findings += f"### {icon} {category.upper()} Compliance\n\n"

                for issue in results['issues']:
                    severity_icon = self.severity_colors[issue['severity']]
                    findings += f"**{severity_icon} {issue['severity']} Priority:** {issue['issue']}\n"
                    findings += f"*Description:* {issue['description']}\n\n"

                # Add passed items
                if results['passed']:
                    findings += "**✅ Passed Requirements:**\n"
                    for passed in results['passed']:
                        findings += f"- {passed}\n"
                    findings += "\n"

        return findings

    def generate_priority_matrix(self, compliance_results):
        """Generate priority matrix for issues"""
        matrix = "## 📈 Priority Matrix\n\n"

        # Group issues by severity
        issues_by_severity = {'High': [], 'Medium': [], 'Low': []}

        for issue in compliance_results['issues']:
            issues_by_severity[issue['severity']].append(issue)

        for severity in ['High', 'Medium', 'Low']:
            if issues_by_severity[severity]:
                icon = self.severity_colors[severity]
                matrix += f"### {icon} {severity} Priority Issues ({len(issues_by_severity[severity])})\n\n"

                for i, issue in enumerate(issues_by_severity[severity], 1):
                    category_icon = self.category_icons.get(issue['category'], '📌')
                    matrix += f"{i}. {category_icon} **{issue['issue']}** ({issue['category']})\n"
                    matrix += f"   - {issue['description']}\n\n"

        return matrix

    def generate_implementation_roadmap(self, compliance_results):
        """Generate implementation roadmap"""
        roadmap = "## 🗺️ Implementation Roadmap\n\n"

        # Create phases based on severity and complexity
        phases = {
            'Phase 1 (Immediate - 1-2 weeks)': [],
            'Phase 2 (Short-term - 1-2 months)': [],
            'Phase 3 (Long-term - 3-6 months)': []
        }

        # Categorize recommendations
        for recommendation in compliance_results['recommendations']:
            if recommendation.get('priority') == 'High':
                phases['Phase 1 (Immediate - 1-2 weeks)'].append(recommendation)
            elif recommendation.get('priority') == 'Medium':
                phases['Phase 2 (Short-term - 1-2 months)'].append(recommendation)
            else:
                phases['Phase 3 (Long-term - 3-6 months)'].append(recommendation)

        for phase, recommendations in phases.items():
            if recommendations:
                roadmap += f"### {phase}\n\n"
                for i, rec in enumerate(recommendations, 1):
                    category_icon = self.category_icons.get(rec['category'], '📌')
                    roadmap += f"{i}. {category_icon} {rec['recommendation']}\n"
                roadmap += "\n"

        return roadmap

    def generate_technical_details(self, website_data, compliance_results):
        """Generate technical details section"""
        details = "## 🔧 Technical Details\n\n"

        details += f"### Website Structure Analysis\n"
        details += f"- **Total Images:** {len(website_data['images'])}\n"
        details += f"- **Total Forms:** {len(website_data['forms'])}\n"
        details += f"- **Total Links:** {len(website_data['links'])}\n"
        details += f"- **Heading Structure:** {len(website_data['headings'])} headings\n"
        details += f"- **Meta Tags:** {len(website_data['meta_tags'])} tags found\n\n"

        # Image analysis
        if website_data['images']:
            images_without_alt = [img for img in website_data['images'] if not img['alt']]
            details += f"### Image Analysis\n"
            details += f"- **Images without alt text:** {len(images_without_alt)}\n"
            details += f"- **Alt text coverage:** {((len(website_data['images']) - len(images_without_alt)) / len(website_data['images']) * 100):.1f}%\n\n"

        # Form analysis
        if website_data['forms']:
            details += f"### Form Analysis\n"
            for i, form in enumerate(website_data['forms'], 1):
                details += f"**Form {i}:**\n"
                details += f"- Method: {form['method']}\n"
                details += f"- Action: {form['action'] or 'Same page'}\n"
                details += f"- Input fields: {len(form['inputs'])}\n\n"

        return details

    def format_html_report(self, report_data):
        """Format report as HTML for better presentation"""
        html_template = """
        <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; line-height: 1.6; color: #333; max-width: 1200px; margin: 0 auto; padding: 20px;">
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; border-radius: 10px; margin-bottom: 30px; text-align: center;">
                <h1 style="margin: 0; font-size: 2.5em;">🔍 Website Compliance Audit Report</h1>
                <p style="margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;">Comprehensive Analysis & Recommendations</p>
            </div>

            <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; margin-bottom: 30px;">
                <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #28a745;">
                    <h3 style="margin: 0 0 10px 0; color: #28a745;">Compliance Score</h3>
                    <div style="font-size: 2em; font-weight: bold;">{score}%</div>
                </div>
                <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #dc3545;">
                    <h3 style="margin: 0 0 10px 0; color: #dc3545;">Total Issues</h3>
                    <div style="font-size: 2em; font-weight: bold;">{total_issues}</div>
                </div>
                <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #fd7e14;">
                    <h3 style="margin: 0 0 10px 0; color: #fd7e14;">High Priority</h3>
                    <div style="font-size: 2em; font-weight: bold;">{high_issues}</div>
                </div>
            </div>

            <div style="background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); margin-bottom: 20px;">
                {content}
            </div>
        </div>
        """

        # Convert markdown to basic HTML formatting
        content = report_data.replace('\n## ', '\n<h2>').replace('\n### ', '\n<h3>')
        content = content.replace('**', '<strong>').replace('**', '</strong>')
        content = content.replace('\n- ', '\n<li>').replace('\n\n', '</p><p>')
        content = f"<p>{content}</p>"

        return html_template.format(
            content=content,
            score=85,  # This would be dynamic
            total_issues=12,  # This would be dynamic
            high_issues=3  # This would be dynamic
        )

    def export_report_formats(self, report_data, base_filename="audit_report"):
        """Export report in multiple formats"""
        formats = {}

        # Generate markdown format
        markdown_content = ""
        for section_name, section_content in report_data.items():
            if isinstance(section_content, str):
                markdown_content += f"\n{section_content}\n"
            elif isinstance(section_content, dict):
                markdown_content += f"\n## {section_name.replace('_', ' ').title()}\n"
                for key, value in section_content.items():
                    if isinstance(value, str):
                        markdown_content += f"\n### {key.replace('_', ' ').title()}\n{value}\n"

        formats['markdown'] = markdown_content
        formats['html'] = self.format_html_report(markdown_content)

        # Generate JSON format for API consumption
        formats['json'] = json.dumps(report_data, indent=2)

        return formats

# Initialize report generator
report_generator = ReportGenerator()

print("✅ Report Generator initialized successfully!")

✅ Report Generator initialized successfully!


In [8]:
# AI-Powered Site Auditor - Section 7: Gradio Interface and Main Application

class SiteAuditorApp:
    def __init__(self):
        self.scraper = WebsiteScraper()
        self.compliance_checker = ComplianceChecker()
        self.report_generator = ReportGenerator()
        self.ai_analyzer = None
        self.rag_module = rag_module  # From previous section

    def initialize_ai(self, api_key):
        """Initialize AI analyzer with provided API key"""
        if api_key and api_key.strip():
            try:
                self.ai_analyzer = AIAnalyzer(api_key.strip())
                return "✅ AI Analyzer initialized successfully!"
            except Exception as e:
                return f"❌ Failed to initialize AI: {str(e)}"
        return "⚠️ No API key provided"

    def audit_website(self, url, api_key="", enable_ai=True, enable_rag=True, progress=gr.Progress()):
        """Main audit function"""
        try:
            # Progress tracking
            progress(0.1, desc="Validating URL...")

            if not url or not url.strip():
                return "❌ Please enter a valid URL", "", ""

            url = url.strip()
            if not url.startswith(('http://', 'https://')):
                url = 'https://' + url

            # Initialize AI if requested and API key provided
            ai_status = ""
            if enable_ai and api_key:
                ai_status = self.initialize_ai(api_key)

            # Step 1: Scrape website
            progress(0.2, desc="Scraping website content...")
            website_data, error = self.scraper.fetch_website_content(url)

            if error:
                return f"❌ Error: {error}", "", ""

            # Step 2: Run compliance checks
            progress(0.4, desc="Analyzing compliance...")
            compliance_results = self.compliance_checker.analyze_compliance(website_data)

            # Step 3: Generate AI analysis (if enabled)
            ai_analysis = None
            if enable_ai and self.ai_analyzer:
                progress(0.6, desc="Generating AI insights...")
                try:
                    ai_analysis = self.ai_analyzer.analyze_website_with_ai(website_data, compliance_results)
                except Exception as e:
                    ai_status += f"\n⚠️ AI analysis failed: {str(e)}"

            # Step 4: Get RAG recommendations (if enabled)
            rag_recommendations = None
            if enable_rag and self.rag_module:
                progress(0.8, desc="Fetching regulatory guidance...")
                try:
                    rag_recommendations = self.rag_module.get_contextual_recommendations(compliance_results['issues'])
                except Exception as e:
                    print(f"RAG recommendations failed: {str(e)}")

            # Step 5: Generate comprehensive report
            progress(0.9, desc="Generating report...")
            report_data = self.report_generator.generate_comprehensive_report(
                website_data, compliance_results, ai_analysis, rag_recommendations
            )

            # Format outputs
            progress(1.0, desc="Finalizing report...")

            # Main report (Markdown)
            main_report = f"{report_data['executive_summary']}\n{report_data['detailed_findings']}\n{report_data['priority_matrix']}"

            # AI Insights (if available)
            ai_insights_text = ""
            if ai_analysis:
                ai_insights_text = f"""
## 🤖 AI-Powered Insights

### Key Insights
{ai_analysis.get('ai_insights', 'No insights available')}

### Priority Recommendations
{ai_analysis.get('priority_recommendations', 'No recommendations available')}

### Improvement Suggestions
{ai_analysis.get('improvement_suggestions', 'No suggestions available')}
"""

            # Implementation roadmap
            implementation_roadmap = report_data['implementation_roadmap']

            # Add AI status to main report if relevant
            if ai_status:
                main_report = f"{ai_status}\n\n{main_report}"

            return main_report, ai_insights_text, implementation_roadmap

        except Exception as e:
            return f"❌ Audit failed: {str(e)}", "", ""

    def update_rag_content(self, progress=gr.Progress()):
        """Update RAG content with latest regulations"""
        if not self.rag_module:
            return "❌ RAG module not available"

        try:
            progress(0.2, desc="Fetching GDPR content...")
            progress(0.5, desc="Fetching WCAG guidelines...")
            progress(0.8, desc="Fetching ADA guidance...")
            self.rag_module.update_regulatory_content()
            return "✅ Regulatory content updated successfully!"
        except Exception as e:
            return f"❌ Failed to update regulatory content: {str(e)}"

# Initialize the app
app = SiteAuditorApp()

print("✅ Site Auditor App initialized!")
print("💡 Ready to build the Gradio interface")



✅ Site Auditor App initialized!
💡 Ready to build the Gradio interface


In [9]:
# AI-Powered Site Auditor - Section 8: Gradio Interface Definition

with gr.Blocks() as demo:
    gr.Markdown("# AI-Powered Website Compliance Auditor")
    gr.Markdown("Enter a website URL to audit its compliance with GDPR, Accessibility (WCAG/ADA), SEO, and Security standards.")

    with gr.Row():
        url_input = gr.Textbox(label="Website URL", placeholder="e.g., https://www.example.com")
        audit_button = gr.Button("Run Audit")

    with gr.Accordion("⚙️ Advanced Settings", open=False):
        api_key_input = gr.Textbox(label="Gemini API Key (Optional, for AI insights)", type="password")
        enable_ai_checkbox = gr.Checkbox(label="Enable AI-Powered Insights", value=True)
        enable_rag_checkbox = gr.Checkbox(label="Enable Regulatory Guidance (RAG)", value=True)
        update_rag_button = gr.Button("Update Regulatory Content (Takes time)")
        rag_update_status = gr.Textbox(label="RAG Update Status", interactive=False)

    with gr.Tabs():
        with gr.TabItem("📊 Audit Report"):
            report_output = gr.Markdown("Report will appear here...")
        with gr.TabItem("🤖 AI Insights"):
            ai_insights_output = gr.Markdown("AI Insights will appear here...")
        with gr.TabItem("🗺️ Implementation Roadmap"):
            roadmap_output = gr.Markdown("Implementation Roadmap will appear here...")

    audit_button.click(
        app.audit_website,
        inputs=[url_input, api_key_input, enable_ai_checkbox, enable_rag_checkbox],
        outputs=[report_output, ai_insights_output, roadmap_output]
    )

    update_rag_button.click(
        app.update_rag_content,
        inputs=[],
        outputs=[rag_update_status]
    )

print("✅ Gradio interface defined!")
print("💡 Ready to launch the app")

✅ Gradio interface defined!
💡 Ready to launch the app


In [ ]:
# AI-Powered Site Auditor - Section 9: Launch Gradio App

# Launch the Gradio interface
if __name__ == "__main__":
    demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4965fd56b7a57abcff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
